# USA Personalized Regression Comparison

This notebook mirrors the structure of `05_regression_analysis` but focuses on the United States as the origin country. We:

1. Recreate the four gravity-model specifications used previously.
2. Generate destination-specific predictions for students originating from the USA.
3. Summarize the relative attractiveness of each destination with a probability table (no map output).

> Note: We reuse the cleaned fact table and replicate the model specifications to ensure consistency with the Italy analysis.


In [8]:
from pathlib import Path

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import display

BASE_DIR = Path("/Users/simonedinato/Documents/Classes/Applied Econometrics/Project")
DATA_DIR = BASE_DIR / "Datasets"

fact_path = DATA_DIR / "07_fact_tables" / "od_fact_table.csv"

fact = pd.read_csv(fact_path)
fact.head()


,indicatorId,origin_country_code,year,students_outbound_total,qualifier,magnitude,origin_country,destination_country_code,destination_country,students_inbound_destination,...,gdp_pc_dest,gdp_dest_ppp_flag,gdp_dest_year_source,gdp_dest_year_distance,gdp_pc_orig,gdp_orig_ppp_flag,gdp_orig_year_source,gdp_orig_year_distance,log_gdp_gap,gdp_pc_ratio
0,OE.5T8.40510,ABW,2018,365.0,NaN,NaN,Aruba,ALB,Albania,1969.00,...,14711.828522,1.0,2018.0,0.0,39419.555703,1.0,2018.0,0.0,-0.985610,0.373211
1,OE.5T8.40510,ABW,2018,365.0,NaN,NaN,Aruba,AND,Andorra,278.00,...,63048.598557,1.0,2018.0,0.0,39419.555703,1.0,2018.0,0.0,0.469644,1.599424
2,OE.5T8.40510,ABW,2018,365.0,NaN,NaN,Aruba,ARE,United Arab Emirates,199958.00,...,68854.969902,1.0,2018.0,0.0,39419.555703,1.0,2018.0,0.0,0.557740,1.746721
3,OE.5T8.40510,ABW,2018,365.0,NaN,NaN,Aruba,ARG,Argentina,109226.25,...,27367.115094,1.0,2018.0,0.0,39419.555703,1.0,2018.0,0.0,-0.364920,0.694252
4,OE.5T8.40510,ABW,2018,365.0,NaN,NaN,Aruba,ARM,Armenia,4598.00,...,15037.045242,1.0,2018.0,0.0,39419.555703,1.0,2018.0,0.0,-0.963745,0.381462


In [9]:
# Prepare data for regression models (reuse logic from Italy notebook)
reg_data = fact.copy()

# Dependent Variable
reg_data["log_students"] = np.log1p(reg_data["students_enrolled"])

# Independent Variables
reg_data["log_earnings_diff"] = np.log(reg_data["earnings_dest"]) - np.log(reg_data["earnings_orig"])
reg_data["log_tuition_diff"] = np.log1p(reg_data["cost_tuition_dest"]) - np.log1p(reg_data["cost_tuition_orig"])
reg_data["log_living_diff"] = np.log1p(reg_data["cost_living_dest"]) - np.log1p(reg_data["cost_living_orig"])
reg_data["log_dist"] = np.log(reg_data["dist"])
reg_data["log_gdp_dest"] = np.log(reg_data["gdp_pc_dest"])

# Restricted Model Variables
reg_data["total_cost_dest"] = reg_data["cost_tuition_dest"] + reg_data["cost_living_dest"]
reg_data["total_cost_orig"] = reg_data["cost_tuition_orig"] + reg_data["cost_living_orig"]
reg_data["roi_dest"] = reg_data["earnings_dest"] / reg_data["total_cost_dest"]
reg_data["roi_orig"] = reg_data["earnings_orig"] / reg_data["total_cost_orig"]
reg_data["log_roi_diff"] = np.log(reg_data["roi_dest"]) - np.log(reg_data["roi_orig"])

regression_cols = [
    "log_students",
    "log_earnings_diff",
    "log_tuition_diff",
    "log_living_diff",
    "log_dist",
    "comlang_off",
    "colony",
    "log_gdp_dest",
    "log_roi_diff",
]

reg_data = reg_data.replace([np.inf, -np.inf], np.nan)
reg_data = reg_data.dropna(subset=regression_cols)
print(f"Regression rows: {len(reg_data)}")


Regression rows: 17759


/Users/simonedinato/Documents/Classes/Applied Econometrics/Project/.venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Model 1: Base Gravity Model
Standard gravity covariates plus destination GDP (no fixed effects).


In [10]:
model1 = smf.ols(
    "log_students ~ log_tuition_diff + log_earnings_diff + log_living_diff + log_dist + comlang_off + colony + log_gdp_dest",
    data=reg_data,
).fit(cov_type="HC1")
model1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           log_students   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     445.6
Date:                Thu, 04 Dec 2025   Prob (F-statistic):               0.00
Time:                        13:55:48   Log-Likelihood:                -33907.
No. Observations:               17759   AIC:                         6.783e+04
Df Residuals:                   17751   BIC:                         6.789e+04
Df Model:                           7                                         
Covariance Type:                  HC1                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8816      0.301      2.926      0.003       0.291       1.472
log_tuition_diff      0.0248      0.003      8.198      0.000       0.019       0.031
log_earnings_diff     0.1098      0.022      4.912      0.000       0.066       0.154
log_living_diff       0.4073      0.025     15.975      0.000       0.357       0.457
log_dist             -0.0067      0.013     -0.513      0.608      -0.032       0.019
comlang_off           0.6846      0.044     15.486      0.000       0.598       0.771
colony                1.0824      0.085     12.731      0.000       0.916       1.249
log_gdp_dest          0.3236      0.024     13.503      0.000       0.277       0.371
==============================================================================
Omnibus:                       95.622   Durbin-Watson:                   1.196
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               77.089
Skew:                          -0.087   Prob(JB):                     1.82e-17
Kurtosis:                       2.729   Cond. No.                         344.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

## Model 2: Origin Fixed Effects
Adds origin-country fixed effects to soak up unobserved heterogeneity.


In [11]:
model2 = smf.ols(
    "log_students ~ log_tuition_diff + log_earnings_diff + log_living_diff + log_dist + comlang_off + colony + C(origin_country_code)",
    data=reg_data,
).fit(cov_type="HC1")
model2.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           log_students   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     292.8
Date:                Thu, 04 Dec 2025   Prob (F-statistic):               0.00
Time:                        13:55:48   Log-Likelihood:                -28855.
No. Observations:               17759   AIC:                         5.784e+04
Df Residuals:                   17693   BIC:                         5.836e+04
Df Model:                          65                                         
Covariance Type:                  HC1                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         1.5395      0.134     11.468      0.000       1.276       1.803
C(origin_country_code)[T.AUS]     3.3468      0.108     30.864      0.000       3.134       3.559
C(origin_country_code)[T.AUT]     2.6304      0.104     25.201      0.000       2.426       2.835
C(origin_country_code)[T.BEL]     3.2382      0.110     29.341      0.000       3.022       3.455
C(origin_country_code)[T.BGD]     2.0247      0.102     19.826      0.000       1.825       2.225
C(origin_country_code)[T.BGR]     1.5934      0.102     15.560      0.000       1.393       1.794
C(origin_country_code)[T.BHR]     0.9682      0.097      9.956      0.000       0.778       1.159
C(origin_country_code)[T.BRA]     3.0407      0.101     30.036      0.000       2.842       3.239
C(origin_country_code)[T.CAN]     4.2750      0.108     39.640      0.000       4.064       4.486
C(origin_country_code)[T.CHE]     4.4275      0.114     38.669      0.000       4.203       4.652
C(origin_country_code)[T.COL]     2.5180      0.102     24.727      0.000       2.318       2.718
C(origin_country_code)[T.CYP]     2.2076      0.104     21.163      0.000       2.003       2.412
C(origin_country_code)[T.DEU]     4.1484      0.102     40.693      0.000       3.949       4.348
C(origin_country_code)[T.DNK]     1.3393      0.100     13.411      0.000       1.144       1.535
C(origin_country_code)[T.DOM]    -0.1210      0.098     -1.230      0.219      -0.314       0.072
C(origin_country_code)[T.DZA]     1.2546      0.103     12.171      0.000       1.053       1.457
C(origin_country_code)[T.ECU]     2.0641      0.103     19.964      0.000       1.861       2.267
C(origin_country_code)[T.EGY]     1.8014      0.101     17.869      0.000       1.604       1.999
C(origin_country_code)[T.ESP]     3.5392      0.105     33.703      0.000       3.333       3.745
C(origin_country_code)[T.FIN]     1.5993      0.100     16.050      0.000       1.404       1.795
C(origin_country_code)[T.FRA]     4.6161      0.107     43.246      0.000       4.407       4.825
C(origin_country_code)[T.GBR]     3.8640      0.109     35.503      0.000       3.651       4.077
C(origin_country_code)[T.GHA]     1.5141      0.104     14.600      0.000       1.311       1.717
C(origin_country_code)[T.GRC]     2.1284      0.100     21.191      0.000       1.932       2.325
C(origin_country_code)[T.HRV]     0.9811      0.100      9.770      0.000       0.784       1.178
C(origin_country_code)[T.HUN]     1.0494      0.101     10.351      0.000       0.851       1.248
C(origin_country_code)[T.IDN]     2.5487      0.102     24.898      0.000       2.348       2.749
C(origin_country_code)[T.IND]     4.9748      0.105     47.260      0.000       4.768       5.181
C(origin_country_code)[T.IRL]     2.7135     

## Model 3: ROI Specification
Replaces the tuition and earnings controls with the ROI differential.


In [12]:
model3 = smf.ols(
    "log_students ~ log_roi_diff + log_dist + comlang_off + colony + C(origin_country_code)",
    data=reg_data,
).fit(cov_type="HC1")
model3.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           log_students   R-squared:                       0.335
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     145.2
Date:                Thu, 04 Dec 2025   Prob (F-statistic):               0.00
Time:                        13:55:48   Log-Likelihood:                -31717.
No. Observations:               17759   AIC:                         6.356e+04
Df Residuals:                   17695   BIC:                         6.406e+04
Df Model:                          63                                         
Covariance Type:                  HC1                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         4.1593      0.162     25.692      0.000       3.842       4.477
C(origin_country_code)[T.AUS]     0.7979      0.120      6.670      0.000       0.563       1.032
C(origin_country_code)[T.AUT]     0.8095      0.123      6.564      0.000       0.568       1.051
C(origin_country_code)[T.BEL]     0.6975      0.123      5.662      0.000       0.456       0.939
C(origin_country_code)[T.BGD]     2.1278      0.121     17.595      0.000       1.891       2.365
C(origin_country_code)[T.BGR]     0.8706      0.122      7.150      0.000       0.632       1.109
C(origin_country_code)[T.BHR]    -0.5644      0.115     -4.919      0.000      -0.789      -0.339
C(origin_country_code)[T.BRA]     2.4130      0.123     19.684      0.000       2.173       2.653
C(origin_country_code)[T.CAN]     1.9352      0.120     16.163      0.000       1.701       2.170
C(origin_country_code)[T.CHE]     0.9131      0.124      7.383      0.000       0.671       1.156
C(origin_country_code)[T.COL]     2.0069      0.121     16.581      0.000       1.770       2.244
C(origin_country_code)[T.CYP]     0.9674      0.121      7.973      0.000       0.730       1.205
C(origin_country_code)[T.DEU]     2.4831      0.121     20.595      0.000       2.247       2.719
C(origin_country_code)[T.DNK]    -0.4471      0.120     -3.737      0.000      -0.682      -0.213
C(origin_country_code)[T.DOM]    -0.5455      0.116     -4.715      0.000      -0.772      -0.319
C(origin_country_code)[T.DZA]     1.2762      0.120     10.595      0.000       1.040       1.512
C(origin_country_code)[T.ECU]     1.4720      0.122     12.017      0.000       1.232       1.712
C(origin_country_code)[T.EGY]     1.7598      0.120     14.632      0.000       1.524       1.995
C(origin_country_code)[T.ESP]     1.5576      0.125     12.458      0.000       1.313       1.803
C(origin_country_code)[T.FIN]     0.0929      0.120      0.776      0.438      -0.142       0.328
C(origin_country_code)[T.FRA]     2.3848      0.122     19.472      0.000       2.145       2.625
C(origin_country_code)[T.GBR]     1.4390      0.122     11.774      0.000       1.199       1.679
C(origin_country_code)[T.GHA]     1.3127      0.123     10.701      0.000       1.072       1.553
C(origin_country_code)[T.GRC]     1.3530      0.122     11.048      0.000       1.113       1.593
C(origin_country_code)[T.HRV]     0.0360      0.119      0.301      0.763      -0.198       0.270
C(origin_country_code)[T.HUN]     0.3069      0.121      2.544      0.011       0.070       0.543
C(origin_country_code)[T.IDN]     2.1747      0.121     17.934      0.000       1.937       2.412
C(origin_country_code)[T.IND]     4.4479      0.124     35.769      0.000       4.204       4.692
C(origin_country_code)[T.IRL]     0.4598     

## Model 4: Parsimonious Gravity Model
Focuses on structural gravity elements (GDP, distance, cultural ties).


In [13]:
model4 = smf.ols(
    "log_students ~ log_gdp_dest + log_dist + comlang_off + colony + C(origin_country_code)",
    data=reg_data,
).fit(cov_type="HC1")
model4.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           log_students   R-squared:                       0.412
Model:                            OLS   Adj. R-squared:                  0.410
Method:                 Least Squares   F-statistic:                     208.2
Date:                Thu, 04 Dec 2025   Prob (F-statistic):               0.00
Time:                        13:55:48   Log-Likelihood:                -30618.
No. Observations:               17759   AIC:                         6.136e+04
Df Residuals:                   17695   BIC:                         6.186e+04
Df Model:                          63                                         
Covariance Type:                  HC1                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -5.5907      0.225    -24.833      0.000      -6.032      -5.149
C(origin_country_code)[T.AUS]     0.2411      0.108      2.230      0.026       0.029       0.453
C(origin_country_code)[T.AUT]     0.9227      0.115      8.016      0.000       0.697       1.148
C(origin_country_code)[T.BEL]     1.3308      0.117     11.339      0.000       1.101       1.561
C(origin_country_code)[T.BGD]     1.7155      0.111     15.487      0.000       1.498       1.933
C(origin_country_code)[T.BGR]     0.9541      0.113      8.439      0.000       0.732       1.176
C(origin_country_code)[T.BHR]    -0.5699      0.106     -5.373      0.000      -0.778      -0.362
C(origin_country_code)[T.BRA]     2.0967      0.112     18.655      0.000       1.876       2.317
C(origin_country_code)[T.CAN]     1.4969      0.109     13.684      0.000       1.283       1.711
C(origin_country_code)[T.CHE]     1.4022      0.117     11.944      0.000       1.172       1.632
C(origin_country_code)[T.COL]     1.7482      0.111     15.743      0.000       1.531       1.966
C(origin_country_code)[T.CYP]     1.0060      0.113      8.925      0.000       0.785       1.227
C(origin_country_code)[T.DEU]     2.5179      0.113     22.360      0.000       2.297       2.739
C(origin_country_code)[T.DNK]    -0.3483      0.112     -3.120      0.002      -0.567      -0.129
C(origin_country_code)[T.DOM]    -0.7167      0.107     -6.693      0.000      -0.927      -0.507
C(origin_country_code)[T.DZA]     1.2286      0.113     10.878      0.000       1.007       1.450
C(origin_country_code)[T.ECU]     1.0040      0.111      9.078      0.000       0.787       1.221
C(origin_country_code)[T.EGY]     1.7073      0.111     15.382      0.000       1.490       1.925
C(origin_country_code)[T.ESP]     1.4976      0.116     12.950      0.000       1.271       1.724
C(origin_country_code)[T.FIN]     0.1443      0.111      1.297      0.195      -0.074       0.362
C(origin_country_code)[T.FRA]     2.2742      0.112     20.289      0.000       2.055       2.494
C(origin_country_code)[T.GBR]     1.2089      0.112     10.806      0.000       0.990       1.428
C(origin_country_code)[T.GHA]     0.7183      0.110      6.529      0.000       0.503       0.934
C(origin_country_code)[T.GRC]     1.4470      0.114     12.727      0.000       1.224       1.670
C(origin_country_code)[T.HRV]     0.1738      0.111      1.571      0.116      -0.043       0.391
C(origin_country_code)[T.HUN]     0.4385      0.112      3.918      0.000       0.219       0.658
C(origin_country_code)[T.IDN]     1.7834      0.112     15.941      0.000       1.564       2.003
C(origin_country_code)[T.IND]     3.9672      0.112     35.490      0.000       3.748       4.186
C(origin_country_code)[T.IRL]     0.5747     

## Personalized Estimation: United States Origin
We now compare the four models for a student originating from the United States. We reuse the same destination list from the Italy analysis to keep results comparable.


In [14]:
# User Profile Data (Italy - Data Science)
user_origin = "USA"
user_tuition = 60000
user_earnings = 165000
user_living = 55000
user_total_cost = user_tuition + user_living
user_roi = user_earnings / user_total_cost

# Select Top Destinations (Expanded List)
# Added: ESP, NLD, DNK, NOR, SWE, CHE, CHN, JPN, KOR, ARE, ITA
destinations = [
    "USA", "GBR", "DEU", "FRA", "CAN", 
    "ESP", "NLD", "DNK", "NOR", "SWE", "CHE", 
    "CHN", "JPN", "KOR", "ARE", "ITA"
]

# --- FETCH DATA FROM RAW FACT TABLE ---
# We use 'fact' instead of 'reg_data' because 'reg_data' dropped rows with missing values
# We filter for ITA origin and the selected destinations
personal_data = fact[
    (fact["origin_country_code"] == user_origin) &
    (fact["destination_country_code"].isin(destinations))
].copy()

# Filter for the latest year available for each destination
if "year" in personal_data.columns:
    personal_data = personal_data.sort_values("year", ascending=False).drop_duplicates(subset=["destination_country_code"])

# Ensure all structural variables are present (fill from reg_data logic if needed)
# The raw 'fact' table has 'gdp_pc_dest', 'dist', 'comlang_off', 'colony'
# We need to create the log variables

# --- INJECT MISSING DATA (WEB SCRAPED) ---
# Dictionary of missing data
# Format: Country Code: {col: value}
missing_data_map = {
    "CHN": {
        "cost_tuition_dest": 6500,   # ~Avg for Master's
        "earnings_dest": 53000,      # ~Entry Level Data Scientist
        "cost_living_dest": 12000    # ~Major city student living
    },
    "KOR": {
        "cost_tuition_dest": 13000,  # ~Avg Private Uni
        "earnings_dest": 67000,      # ~Entry Level
        "cost_living_dest": 9000     # ~Student avg
    },
    "ARE": {
        "cost_tuition_dest": 25000,  # ~Avg International Uni
        "cost_living_dest": 18000    # ~Mid-range student living
        # Earnings already in dataset (~73k)
    },
    "USA": {
        "dist": 1,              # Hardcoded small distance for internal flow
        "cost_tuition_dest": user_tuition,
        "earnings_dest": user_earnings,
        "cost_living_dest": user_living
    }
}

# Apply the manual data
for country, data in missing_data_map.items():
    mask = personal_data["destination_country_code"] == country
    if mask.any():
        for col, val in data.items():
            personal_data.loc[mask, col] = val
    else:
        print(f"Warning: {country} not found in raw fact rows.")

# -----------------------------------------

# Now calculate the regression variables
# Dependent Variable (placeholder, not needed for prediction but good for consistency)
personal_data["log_students"] = np.log1p(personal_data["students_enrolled"])

# Independent Variables
personal_data["log_earnings_diff"] = np.log(personal_data["earnings_dest"]) - np.log(user_earnings)
personal_data["log_tuition_diff"] = np.log1p(personal_data["cost_tuition_dest"]) - np.log1p(user_tuition)
personal_data["log_living_diff"] = np.log1p(personal_data["cost_living_dest"]) - np.log1p(user_living)
personal_data["log_dist"] = np.log(personal_data["dist"])
personal_data["log_gdp_dest"] = np.log(personal_data["gdp_pc_dest"])

# Restricted Model Variables
personal_data["total_cost_dest"] = personal_data["cost_tuition_dest"] + personal_data["cost_living_dest"]
personal_data["roi_dest"] = personal_data["earnings_dest"] / personal_data["total_cost_dest"]
personal_data["log_roi_diff"] = np.log(personal_data["roi_dest"]) - np.log(user_roi)

# Drop any rows that STILL have NaNs in the required columns (e.g. if we missed some data)
# But we want to keep as many as possible.
# Model 4 only needs GDP, Dist, Culture.
# Model 2 needs Tuition, Earnings.

# Run Predictions
# We handle NaNs by filling with 0 or dropping, but let's try to predict where possible.
# If a row has NaN for a model's features, predict() will return NaN.

personal_data["pred_m1"] = model1.predict(personal_data)
personal_data["pred_m2"] = model2.predict(personal_data)
personal_data["pred_m3"] = model3.predict(personal_data)
personal_data["pred_m4"] = model4.predict(personal_data)

# Convert log predictions back to student counts (exp - 1)
results = personal_data[["destination_country_code"]].copy()

for col in ["pred_m1", "pred_m2", "pred_m3", "pred_m4"]:
    # Calculate raw counts
    results[col + "_count"] = np.expm1(personal_data[col])
    
    # Calculate Probabilities (Weights)
    # The probability of choosing destination j is Count_j / Sum(Counts)
    # We ignore NaNs in the sum
    total_flow = results[col + "_count"].sum()
    results[col + "_prob"] = results[col + "_count"] / total_flow

# Save Results to CSV for Visualization
# FIX: Use ../Datasets because notebook runs in scripts/
results.to_csv("../Datasets/personalized_predictions.csv", index=False)
print("Results saved to ../Datasets/personalized_predictions_USA.csv")

# Display Results as Probabilities
print("Estimated Probability of Choosing Each Destination (Weights):")
cols_prob = ["destination_country_code", "pred_m1_prob", "pred_m2_prob", "pred_m3_prob", "pred_m4_prob"]
display(results[cols_prob].set_index("destination_country_code").style.format({
    "pred_m1_prob": "{:.1%}",
    "pred_m2_prob": "{:.1%}",
    "pred_m3_prob": "{:.1%}",
    "pred_m4_prob": "{:.1%}"
}).background_gradient(cmap="Blues", axis=0))

Results saved to ../Datasets/personalized_predictions_USA.csv
Estimated Probability of Choosing Each Destination (Weights):


,pred_m1_prob,pred_m2_prob,pred_m3_prob,pred_m4_prob
destination_country_code,,,,
USA,11.6%,41.4%,11.5%,2.6%
FRA,13.0%,6.7%,11.9%,12.6%
ARE,6.0%,7.4%,5.6%,6.3%
CAN,5.1%,5.6%,6.0%,4.1%
CHE,3.2%,2.5%,3.5%,2.7%
CHN,3.3%,3.9%,4.7%,2.8%
DNK,4.1%,2.1%,3.7%,6.1%
ESP,11.2%,4.8%,11.4%,11.4%
DEU,3.8%,2.0%,3.8%,5.6%
